# Movie Recommender - IMB Movie Datset (Kaggle)

In [4]:
pip install scipy

In [5]:
#import necessary labraries:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [6]:
data = pd.read_csv('Genre Classification Dataset/train_data.txt', delimiter=':::', engine='python', header=None)

In [7]:
data.head()

,0,1,2,3
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [8]:
data.isnull().sum()
data.dropna(inplace=True)


In [9]:
column_names =['movieId', 'title', 'genre', 'description']
data.columns = column_names
print(data.head())

   movieId                               title       genre  \
0        1       Oscar et la dame rose (2009)       drama    
1        2                       Cupid (1997)    thriller    
2        3   Young, Wild and Wonderful (1980)       adult    
3        4              The Secret Sin (1915)       drama    
4        5             The Unrecovered (2007)       drama    

                                         description  
0   Listening in to a conversation between his do...  
1   A brother and sister with a past incestuous r...  
2   As the bus empties the students for their fie...  
3   To help their unemployed father make ends mee...  
4   The film's title refers not only to the un-re...  


In [10]:
data['movie_features']=data['genre']+ ' ' + data['description']

In [11]:
tf=TfidfVectorizer(stop_words='english')
#Fit and form the movie_feature 
tf_matrix = tf.fit_transform(data['movie_features'])
print(tf_matrix.shape) #prints the shape of TF-IDF matrix

(54214, 120800)


In [12]:
#convert to a sparse matrix
tf_sparse = csr_matrix(tf_matrix)
#Use NearsestNeighbors for the approx recommendations
nn=NearestNeighbors(n_neighbors=11, metric='cosine',n_jobs=-1)
nn.fit(tf_sparse)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=11)

In [22]:
#get movie index
def get_movie_index(title):
    title=title.strip()
    indices= data.index[data['title'].str.strip() == title].tolist()

    if not indices:
        raise ValueError(f"Movie title {title} not found in dataset.")
    return indices[0]
# print(data['title'].tolist())#Printing titles to verify

In [27]:
#Get_recommendations
def get_recommendations(title, nn_model =nn):
    try: 
        idx =get_movie_index(title)
        distances, indices=nn_model.kneighbors(tf_sparse[idx])
        similar_movies = [data ['title'].iloc[i] for i in indices.flatten()[1:]] #excluding the movie 
        return similar_movies
    except ValueError as e:
        print (e)
        return[]

In [37]:
#Testing the recommender using test 'Oscar et la dame rose (2009)'
recommendations = get_recommendations(' Cupid (1997)')
recommendation_df = pd.DataFrame(recommendations, columns=['Recommended Movies'])
recommendation_df.index = recommendation_df.index + 1
print (recommendation_df)

                                   Recommended Movies
1             Candyman: Farewell to the Flesh (1995) 
2                                   Mysterium (????) 
3                                   "Teoneol" (2017) 
4    Women on the Land - Creating Conscious Commun...
5                                Un alma pura (1965) 
6                            Silent Vengeance (2011) 
7                                    Typhoeus (2012) 
8                                Disposable (2017/I) 
9                             "Dual Suspects" (2010) 
10                                Nameless (2016/II) 
